In [ ]:
!wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
!tar -zxvf kftt-data-1.0.tar.gz

--2021-07-03 08:21:30--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘kftt-data-1.0.tar.gz’

kftt-data-1.0.tar.g 100%[===================>]  94.65M   106MB/s    in 0.9s    

2021-07-03 08:21:31 (106 MB/s) - ‘kftt-data-1.0.tar.gz’ saved [99246893/99246893]

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/tok/kyoto-dev.ja
kftt-data-1.0/data

In [ ]:
!pip3 install fairseq

     |████████████████████████████████| 1.7MB 28.5MB/s 
     |████████████████████████████████| 153kB 54.3MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 112kB 62.4MB/s 
     |████████████████████████████████| 81kB 13.0MB/s 
     |████████████████████████████████| 645kB 48.4MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp37-none-any.whl size=141231 sha256=26f05b1dd4f1e2b998423642950c1b0bbf91f525eaa859b8c02bb05300abc88d
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built antlr4-python3-runtime
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!mkdir data-bin
!mkdir data-bin/kftt.ja-en

In [ ]:
!pip install sentencepiece
import sentencepiece as spm
import re
spm.SentencePieceTrainer.Train('--input=kftt-data-1.0/data/orig/kyoto-train.ja --model_prefix=kyoto_ja --vocab_size=16000 --character_coverage=1.0')

sp = spm.SentencePieceProcessor()
sp.Load('kyoto_ja.model')

for src, dst in [
    ('kftt-data-1.0/data/orig/kyoto-train.ja', 'bpe-kyoto-train.ja'),
    ('kftt-data-1.0/data/orig/kyoto-dev.ja', 'bpe-kyoto-dev.ja'),
    ('kftt-data-1.0/data/orig/kyoto-test.ja', 'bpe-kyoto-test.ja'),
]:
  with open(src) as f:
    with open(dst, 'w') as g:
          for x in f:
              x = x.strip()
              x = re.sub(r'\s+', ' ', x)
              x = sp.encode_as_pieces(x)
              x = ' '.join(x)
              print(x, file=g)

     |████████████████████████████████| 1.2MB 38.6MB/s 


In [ ]:
!pip install subword-nmt
!subword-nmt learn-bpe -s 16000 < kftt-data-1.0/data/tok/kyoto-train.en > kyoto_en.codes
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/tok/kyoto-train.en > bpe-kyoto-train.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/tok/kyoto-dev.en > bpe-kyoto-dev.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/tok/kyoto-test.en > bpe-kyoto-test.en

In [ ]:
!fairseq-preprocess \
  --source-lang ja --target-lang en \
  --trainpref bpe-kyoto-train \
  --validpref bpe-kyoto-dev \
  --testpref bpe-kyoto-test \
  --destdir data95 \
  --thresholdsrc 5 \
  --thresholdtgt 5 \
  --workers 20

2021-07-03 08:27:08 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data95', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='ja', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref='bpe-kyoto-test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=5, thresholdtgt=5, tokenizer=None, tpu=False, trainpref='b

In [ ]:
!pip install sacremoses
!CUDA_VISIBLE_DEVICES=0 fairseq-train data95 \
  --fp16 \
  --save-dir save95 \
  --max-epoch 1 \
  --arch transformer --share-decoder-input-output-embed \
  --optimizer adam --clip-norm 1.0 \
  --lr 0.001 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --update-freq 1 \
  --dropout 0.2 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-remove-bpe \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric > knock95.log

     |████████████████████████████████| 901kB 28.8MB/s 
epoch 001: 100% 3778/3779 [09:25<00:00,  7.18it/s, loss=7.402, nll_loss=6.291, ppl=78.31, wps=24148.4, ups=6.63, wpb=3641.8, bsz=108.3, num_updates=3700, lr=0.000925, gnorm=0.826, clip=8, loss_scale=8, train_wall=15, wall=555]
epoch 001 | valid on 'valid' subset:   0% 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)

epoch 001 | valid on 'valid' subset:   7% 1/15 [00:00<00:12,  1.11it/s]
epoch 001 | valid on 'valid' subset:

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-interactive data95 \
  --path save95/checkpoint_best.pt < bpe-kyoto-test.ja \
  | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 95.out

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
!fairseq-score --sys 95.out --ref kftt-data-1.0/data/tok/kyoto-test.en

Namespace(ignore_case=False, order=4, ref='kftt-data-1.0/data/tok/kyoto-test.en', sacrebleu=False, sentence_bleu=False, sys='95.out')
BLEU4 = 1.06, 10.4/1.6/0.4/0.2 (BP=1.000, ratio=1.697, syslen=45366, reflen=26734)


In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-interactive data95 \
  --path save95/checkpoint_best.pt \
  --beam 1  < bpe-kyoto-test.ja \
  | grep '^H' | cut -f3 > 95.1.out

!CUDA_VISIBLE_DEVICES=0 fairseq-interactive data95 \
  --path save95/checkpoint_best.pt \
  --beam 20 < bpe-kyoto-test.ja \
  | grep '^H' | cut -f3 > 95.20.out

!fairseq-score --sys 95.1.out --ref kftt-data-1.0/data/tok/kyoto-test.en
!fairseq-score --sys 95.20.out --ref kftt-data-1.0/data/tok/kyoto-test.en

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /

# knock96

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 133kB 27.2MB/s 


In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train data95 \
  --fp16 \
  --tensorboard-logdir log96 \
  --save-dir save96 \
  --max-epoch 1 \
  --arch transformer --share-decoder-input-output-embed \
  --optimizer adam --clip-norm 1.0 \
  --lr 0.001 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --update-freq 1 \
  --dropout 0.2 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-remove-bpe \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric > knock96.log

epoch 001: 100% 3778/3779 [09:34<00:00,  7.23it/s, loss=7.402, nll_loss=6.291, ppl=78.31, wps=24188.7, ups=6.64, wpb=3641.8, bsz=108.3, num_updates=3700, lr=0.000925, gnorm=0.826, clip=8, loss_scale=8, train_wall=15, wall=564]
epoch 001 | valid on 'valid' subset:   0% 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)

epoch 001 | valid on 'valid' subset:   7% 1/15 [00:00<00:12,  1.09it/s]
epoch 001 | valid on 'valid' subset:  13% 2/15 [00:01<00:10,  1.21it/s]
epoch 001 | valid on

# knock97

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train data95 \
  --fp16 \
  --save-dir save97_1 \
  --max-epoch 1 \
  --arch transformer --share-decoder-input-output-embed \
  --optimizer adam --clip-norm 1.0 \
  --lr 0.002 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --update-freq 1 \
  --dropout 0.1 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-remove-bpe \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric > knock97_1.log

epoch 001: 100% 3778/3779 [09:27<00:00,  7.42it/s, loss=10.77, nll_loss=10.224, ppl=1195.84, wps=24930.5, ups=6.82, wpb=3656.1, bsz=107.7, num_updates=3700, lr=0.004625, gnorm=0.298, clip=1, loss_scale=1, train_wall=15, wall=557]
epoch 001 | valid on 'valid' subset:   0% 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)

epoch 001 | valid on 'valid' subset:   7% 1/15 [00:00<00:13,  1.05it/s]
epoch 001 | valid on 'valid' subset:  13% 2/15 [00:01<00:11,  1.14it/s]
epoch 001 | valid

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train data95 \
  --fp16 \
  --save-dir save97_2 \
  --max-epoch 1 \
  --arch transformer --share-decoder-input-output-embed \
  --optimizer adam --clip-norm 1.0 \
  --lr 0.0001 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --update-freq 1 \
  --dropout 0.3 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-remove-bpe \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric > knock97_2.log

epoch 001: 100% 3778/3779 [09:37<00:00,  7.13it/s, loss=8.554, nll_loss=7.65, ppl=200.91, wps=24058.5, ups=6.6, wpb=3647.4, bsz=107.6, num_updates=3700, lr=9.25e-05, gnorm=1.526, clip=100, loss_scale=16, train_wall=15, wall=567]
epoch 001 | valid on 'valid' subset:   0% 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)

epoch 001 | valid on 'valid' subset:   7% 1/15 [00:00<00:11,  1.19it/s]
epoch 001 | valid on 'valid' subset:  13% 2/15 [00:01<00:10,  1.30it/s]
epoch 001 | valid 

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-interactive data95 \
  --path save97_1/checkpoint_best.pt < bpe-kyoto-test.ja \
  | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 97_1.out

!CUDA_VISIBLE_DEVICES=0 fairseq-interactive data95 \
  --path save97_2/checkpoint_best.pt < bpe-kyoto-test.ja \
  | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 97_2.out

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /

In [ ]:
!fairseq-score --sys 97_1.out --ref kftt-data-1.0/data/tok/kyoto-test.en
!fairseq-score --sys 97_2.out --ref kftt-data-1.0/data/tok/kyoto-test.en

Namespace(ignore_case=False, order=4, ref='kftt-data-1.0/data/tok/kyoto-test.en', sacrebleu=False, sentence_bleu=False, sys='97_1.out')
BLEU4 = 0.00, 0.0/0.0/0.0/0.0 (BP=1.000, ratio=8.678, syslen=232000, reflen=26734)
Namespace(ignore_case=False, order=4, ref='kftt-data-1.0/data/tok/kyoto-test.en', sacrebleu=False, sentence_bleu=False, sys='97_2.out')
BLEU4 = 1.17, 14.6/2.2/0.5/0.1 (BP=1.000, ratio=1.482, syslen=39628, reflen=26734)
